# Using the SSC Assistant API

This Python notebook can be used on your local device using VS Code. It contains working code that you can execute locally after cloning or copying. It demonstrates how to use the SSC Assistant API using your given API token and a user Access token (oauth2).

Currently, we use the OAuth2 Client Credentials Flow. We do not support user authentication flows for API calls at this time (although it might still be technically possible). In other words, you will have to authenticate as your app itself; a user will never have to be asked for permissions to log into the app, and there will be no redirect. In order to help us support you better, we strongly recommend not using user authentication flows for the API requests at this time.

## Environment Variables

First, copy the `.env.example` that is adjacent to this file into a new file, `.env`. Then, fill in the variables according to the following instructions.

The variables can be broadly categorized as either SSC Assistant-related, or client application-related

### SSC Assistant-Related Environment Variables

#### API_ENDPOINT

The `API_ENDPOINT` points at the SSC Assistant API server that you would like to query. You will be provided this value by our team upon request.

Example:

```
API_ENDPOINT=http://127.0.0.1:5001
```

#### API_SCOPE

The `API_SCOPE` defines the permissions that your session will contain. The `applicationId` (also called the `clientId`) part is referring to the SSC Assistant's ID. You will be provided this value by our team upon request.

Example:

```
API_SCOPE=api://<applicationId>/.default
```

### Client Application-Related Identifiers

#### APP_ID

This is a credential that is specific to your application. This value can be retrieved from Azure Portal, and is generally provided by an App Administrator who has the correct permissions on Azure Portal.

#### APP_SECRET

This is a credential that is specific to your application. This value can be retrieved from Azure Portal, and is generally provided by an App Administrator who has the correct permissions on Azure Portal.

#### TENANT_ID

This is a credential that is specific to your application. This value can be retrieved from Azure Portal, and is generally provided by an App Administrator who has the correct permissions on Azure Portal.

In [24]:
import os

import jwt
import msal
import requests
from dotenv import load_dotenv

load_dotenv()

api_endpoint = os.getenv("API_ENDPOINT")
api_scope = os.getenv("API_SCOPE")

app_id = os.getenv("APP_ID")
app_secret = os.getenv("APP_SECRET")
tenant_id = os.getenv("TENANT_ID")

app = msal.ConfidentialClientApplication(
    app_id, authority=f"https://login.microsoftonline.com/{tenant_id}",
    client_credential=app_secret)

# for confidential clients, the `.default` scope is the only scope that will work.
result = app.acquire_token_for_client(scopes=[api_scope])

if result and "access_token" in result:
    access_token = result['access_token']
    decoded_token = jwt.decode(access_token, options={"verify_signature": False})
    if "roles" in decoded_token:
        print("roles", decoded_token["roles"])
    else:
        print("No roles in token, might be missing permissions or grant approval..")
else:
    print("oops, no token", result)

roles ['api.access']


## Security

API Is secured and need two different token in order to access it, first an API token (see below) and a user access token (provided by microsoft idp)

Once we have the access_token we simply make a simple call to the SSC Assistant API to ask a one-of question.

### API Token

To do so we need to ensure we have a valid user (above) and a valid token to send in the `X-API-Key` header as part of the `POST` request.

Such token can be crafted for testing as bellow:

**NOTE:** If you are not part of the dev team and you require a real token please contact the SSC Assistant team.

### Confidential Client Application access token

Normally the api relies on User access token, in this case for having 3rd parties access the API they (the application(s) in question) need to be granted the permission to request that scope.

Example of an `az cli` that does such thing. Note that the `api-permission` here is from the appRoles and not the api scopes as oposed to scopes created for user flow.

```bash
az ad app permission add --id <your-app-id> --api <api-app-id> --api-permission <scope-uuid>=Role
az ad app permission grant --id <your-app-id> --api <api-app-id> --scope api.access.app
```

### Extra documentation for the setup of the application scope

* https://stackoverflow.com/questions/77552241/getting-token-but-not-scope-inside-that-token-using-msal-code
* https://learn.microsoft.com/en-us/entra/identity-platform/quickstart-configure-app-access-web-apis
* See this [documentation here](../docs/application-config.md#granting-application-the-role)

# POST /suggest endpoint

The `/suggest` endpoint allows access to our chatbot programmatically for one-off questions.

Please see example below.

In [25]:
import requests
import json

# Assuming you have defined the following variables somewhere in your code:
# api_endpoint, access_token, and api_token

api_token = os.getenv("API_TOKEN")
if not api_token:
    api_token = jwt.encode({'roles': ['suggest',]}, 'secret', algorithm='HS256')

new_question = {
    "query": "What is SSC's content management system?",
    "opts": {
        "language": "en",
        "requester": "mysscplus",
        "system_prompt": "Only respond in pirate speak.",
        "dedupe_citations": True,
    },
}

response = requests.post(
    f"{api_endpoint}/api/1.0/suggest",
    headers={
        "Authorization": f"Bearer {access_token}",
        "X-API-Key": api_token
    },
    json=new_question
)

response_json = response.json()

if response.status_code == 200:
    print(json.dumps(response_json, indent=4))
else:
    print("Error: ", response.text)

ConnectionError: HTTPConnectionPool(host='127.0.0.1', port=5001): Max retries exceeded with url: /api/1.0/suggest (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f58ce3a8710>: Failed to establish a new connection: [Errno 111] Connection refused'))

# Continue Conversation

You can let your users continue chatting about their suggestion with the LLM by redirecting the user to the `/suggest-callback` endpoint in the frontend.

For example:

```html
<!-- As a link -->

<a href="https://assistant.cio-sandbox-ect.ssc-spc.cloud-nuage.canada.ca/suggest-callback?suggestionContextId=<the id of your suggestion context>">Continue this conversation</a>
```


# GET /suggest endpoint

`NOTE: This is an internal API that is used by the SSCA frontend. You probably don't need to use it.`

Now that you have a suggestion, you can retrieve that suggestion from the `GET /suggest` endpoint using the URL parameter `suggestion_id`


In [ ]:
# guard for is response was fail
if not response_json['success'] or 'suggestion_id' not in response_json:
    print("Did not find a successful response. Did POST /suggest call succeed?")
    exit(1)

response = requests.get(
    f"{api_endpoint}/api/1.0/suggest",
    headers={
        "Authorization": f"Bearer {access_token}",
        "X-API-Key": api_token
    },
    params={
        "suggestion_id": response_json['suggestion_id'],
    },
    json=new_question
)

print(json.dumps(response.json(), indent=4))


{
    "citations": [
        {
            "title": "Coming in early 2024: SSC Campus",
            "url": "https://plus.ssc-spc.gc.ca/en/news/corporate-messages/18-12-2023/coming-early-2024-ssc-campus"
        },
        {
            "title": "SSC Campus is here!",
            "url": "https://plus.ssc-spc.gc.ca/en/news/articles/02-04-2024/ssc-campus-here"
        },
        {
            "title": "2024-25 Integrated Business Plan",
            "url": "https://plus.ssc-spc.gc.ca/en/page/2024-25-integrated-business-plan"
        },
        {
            "title": "SSC Campus",
            "url": "https://plus.ssc-spc.gc.ca/en/page/ssc-campus"
        },
        {
            "title": "SSC Campus",
            "url": "https://plus.ssc-spc.gc.ca/en/node/2874"
        },
        {
            "title": "MySSC+ is officially an award-winning site!",
            "url": "https://plus.ssc-spc.gc.ca/en/news/articles/30-03-2023/myssc-officially-award-winning-site"
        }
    ],
    "content": 